# Overlapとmp_term_nameの統合

- 例：abnormal_circulating_glucose_level（連続値）
- 例：male_infertility（カテゴリ値）

In [1]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/g/impc-genes_with_similar_phenotypes'

In [164]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import json

In [165]:
# path_overlap = Path("data", "overlap_ratios_filtered.csv")
# df_overlap = pd.read_csv(path_overlap)

overlap = pickle.load(open("data/overlapeed_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"])
print(df_overlap)

       marker1  marker2  overlap_ratio  overlapped_mp_number  \
0         Dpf2     Aff4          0.200                     4   
1         Dpf2    Fgf15          0.333                     4   
2         Dpf2  Col11a1          0.231                     3   
3         Dpf2   Sptan1          0.154                     4   
4         Dpf2      Nog          0.160                     4   
...        ...      ...            ...                   ...   
128944    Mcm4   Exosc8          1.000                     3   
128945  Zfp407   Washc4          0.750                     3   
128946    Gmnn    Rab35          1.000                     3   
128947   Pola2   Exosc8          1.000                     3   
128948    Mrm2      Jtb          0.600                     3   

                                            overlapped_mp  
0       [abnormal craniofacial morphology, abnormal ki...  
1       [abnormal embryo size, abnormal heart morpholo...  
2       [abnormal limb morphology, abnormal tail mo

In [166]:
# path_marker_effect = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
path_marker_effect = Path("data", "mp_term_name", "male_infertility.csv")
df_marker_effect = pd.read_csv(path_marker_effect)
df_marker_effect = df_marker_effect[["marker_symbol", "effect_size"]].drop_duplicates()


In [167]:
print(df_overlap.head(3))

print(len(df_marker_effect))

df_marker_effect

  marker1  marker2  overlap_ratio  overlapped_mp_number  \
0    Dpf2     Aff4          0.200                     4   
1    Dpf2    Fgf15          0.333                     4   
2    Dpf2  Col11a1          0.231                     3   

                                       overlapped_mp  
0  [abnormal craniofacial morphology, abnormal ki...  
1  [abnormal embryo size, abnormal heart morpholo...  
2  [abnormal limb morphology, abnormal tail morph...  
318


,marker_symbol,effect_size
0,Tex38,1
1,Pou2af3,1
2,Stoml2,1
3,Sbf1,1
4,Cyrib,1
...,...,...
320,Cep19,1
321,Bzw1,1
322,Septin10,1
323,Cfap57,1


In [168]:
# print(df_mp["effect_size"].describe())

In [169]:
# Absolute value of effect size

df_marker_effect.loc[:, "effect_size"] = df_marker_effect["effect_size"].abs()

In [170]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [171]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.25)]

In [172]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_marker_effect['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_marker_effect['marker_symbol']))
]

In [173]:

print(len(df_filtered))

df_filtered.head(3)

369


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
4356,Camsap3,Prdm14,0.280,7,"[abnormal bone structure, decreased bone miner..."
4371,Camsap3,Kash5,0.348,8,"[abnormal bone structure, decreased bone miner..."
4455,Camsap3,Trim37,0.276,8,"[abnormal auditory brainstem response, abnorma..."


### 出力

In [174]:
# Edge list

df_edge = df_filtered[["marker1", "marker2", "overlap_ratio", "overlapped_mp"]]
print(len(df_edge))
print(df_edge.head(3))


369
      marker1 marker2  overlap_ratio  \
4356  Camsap3  Prdm14          0.280   
4371  Camsap3   Kash5          0.348   
4455  Camsap3  Trim37          0.276   

                                          overlapped_mp  
4356  [abnormal bone structure, decreased bone miner...  
4371  [abnormal bone structure, decreased bone miner...  
4455  [abnormal auditory brainstem response, abnorma...  


In [175]:
# JSON形式に変換
edge_json = []
for _, row in df_edge.iterrows():
    edge_json.append({
        "data": {
            "source": row['marker1'],
            "target": row['marker2'],
            "annotation": row['overlapped_mp'],
            "edge_size": row['overlap_ratio']
        }
    })

print(edge_json[:3])

[{'data': {'source': 'Camsap3', 'target': 'Prdm14', 'annotation': ['abnormal bone structure', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'female infertility', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.28}}, {'data': {'source': 'Camsap3', 'target': 'Kash5', 'annotation': ['abnormal bone structure', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'female infertility', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.348}}, {'data': {'source': 'Camsap3', 'target': 'Trim37', 'annotation': ['abnormal auditory brainstem response', 'abnormal bone structure', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'female infertility', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.276}}]


In [176]:
# Nodeの情報を作成
# merge mp term
df_marker1 = df_filtered[["marker1"]]
df_marker2 = df_filtered[["marker2"]]
df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]

df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()

print(df_node)
import pickle
marker_mp = pickle.load(open("data/marker_mp.pkl", "rb"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term"])

df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
print(df_node)

df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

    marker_symbol
0         Camsap3
9          Dnmt3l
16         Hectd3
27         Celsr1
28         Stoml2
..            ...
308        Tbc1d8
315       Tbc1d20
329        Rnf216
344          Cib4
359        Scube1

[136 rows x 1 columns]
    marker_symbol                                            mp_term
0         Camsap3  [abnormal auditory brainstem response, abnorma...
1          Dnmt3l  [female infertility, increased bone mineral de...
2          Hectd3  [abnormal bone structure, decreased body lengt...
3          Celsr1  [abnormal head morphology, abnormal retina mor...
4          Stoml2  [decreased prepulse inhibition, increased circ...
..            ...                                                ...
131        Tbc1d8  [abnormal retina morphology, cataract, male in...
132       Tbc1d20  [abnormal skin morphology, hemorrhage, male in...
133        Rnf216  [decreased lymphocyte cell number, increased m...
134          Cib4  [abnormal eye morphology, abnormal skin morpho...
1

In [177]:
# "Herc1"があるか？
print(df_node[df_node["marker_symbol"] == "Herc1"])

   marker_symbol                                            mp_term  \
73         Herc1  [abnormal auditory brainstem response, abnorma...   

    effect_size  
73            1  


In [178]:
# JSON形式に変換
node_json = []
for _, row in df_node.iterrows():
    node_json.append({
        "data": {
            "id": row['marker_symbol'],
            "label": row['marker_symbol'],
            "annotation": row['mp_term'],
            "node_color": row['effect_size']
        }
    })

print(node_json[:3])

[{'data': {'id': 'Camsap3', 'label': 'Camsap3', 'annotation': ['abnormal auditory brainstem response', 'abnormal bone structure', 'abnormal cornea morphology', 'abnormal lens morphology', 'absent pinna reflex', 'cataract', 'corneal opacity', 'decreased blood urea nitrogen level', 'decreased body length', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased circulating cholesterol level', 'decreased lean body mass', 'female infertility', 'fused cornea and lens', 'increased circulating alkaline phosphatase level', 'increased circulating thyroxine level', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility', 'trunk curl'], 'node_color': 1}}, {'data': {'id': 'Dnmt3l', 'label': 'Dnmt3l', 'annotation': ['female infertility', 'increased bone mineral density', 'increased mean corpuscular hemoglobin', 'male infertility', 'small testis'], 'node_color': 1}}, {'data': {'id': 'Hectd3', 'label': 'Hectd3', 'annotation': ['abnormal bone str

In [179]:
# Combine node and edge

network_json = node_json + edge_json

In [180]:
print(len(network_json))
print(network_json[:3])
print(network_json[-3:])

505
[{'data': {'id': 'Camsap3', 'label': 'Camsap3', 'annotation': ['abnormal auditory brainstem response', 'abnormal bone structure', 'abnormal cornea morphology', 'abnormal lens morphology', 'absent pinna reflex', 'cataract', 'corneal opacity', 'decreased blood urea nitrogen level', 'decreased body length', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased circulating cholesterol level', 'decreased lean body mass', 'female infertility', 'fused cornea and lens', 'increased circulating alkaline phosphatase level', 'increased circulating thyroxine level', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility', 'trunk curl'], 'node_color': 1}}, {'data': {'id': 'Dnmt3l', 'label': 'Dnmt3l', 'annotation': ['female infertility', 'increased bone mineral density', 'increased mean corpuscular hemoglobin', 'male infertility', 'small testis'], 'node_color': 1}}, {'data': {'id': 'Hectd3', 'label': 'Hectd3', 'annotation': ['abnormal bone

In [181]:
# Output as JSON file

with open("data/network.json", "w") as f:
    json.dump(network_json, f, indent=2)

# Debug

In [162]:
for data in node_json:
    if "Herc1" == data["data"]["id"]:
        print(data)

{'data': {'id': 'Herc1', 'label': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'increased circulating sodium level', 'impaired glucose tolerance', 'abnormal retina morphology', 'cataract', 'increased circulating chloride level', 'male infertility', 'decreased bone mineral content', 'increased leukocyte cell number', 'increased total body fat amount', 'decreased bone mineral density', 'decreased lean body mass', 'increased red blood cell distribution width', 'abnormal lens morphology', 'increased fasting circulating glucose level', 'increased circulating amylase level', 'decreased circulating glucose level'], 'node_color': 1}}


In [163]:
for data in edge_json:
    if "Herc1" == data["data"]["source"]:
        print(data)
    if "Herc1" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Camsap3', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'cataract', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.31}}
{'data': {'source': 'Prdm14', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'impaired glucose tolerance', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.273}}
{'data': {'source': 'Kash5', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.286}}
{'data': {'source': 'Trim37', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnor

In [150]:
for data in edge_json:
    if "Stoml2" == data["data"]["source"]:
        print(data)
    if "Stoml2" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Vgf', 'target': 'Stoml2', 'annotation': ['decreased prepulse inhibition', 'increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.25}}
{'data': {'source': 'Fbxo38', 'target': 'Stoml2', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.214}}
{'data': {'source': 'Stoml2', 'target': 'Clcn1', 'annotation': ['decreased prepulse inhibition', 'increased circulating aspartate transaminase level', 'increased circulating bilirubin level', 'male infertility'], 'edge_size': 0.182}}
{'data': {'source': 'Stoml2', 'target': 'Lepr', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.073}}
{'data': {'source': 'Stoml2', 'target': 'Atp13a3', 'annotation': ['increased circulating alanine tra